# Generating conformers in RDKit

In [1]:
import os
import glob
import py3Dmol
import numpy as np
%matplotlib inline 

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import rdBase
print(rdBase.rdkitVersion)

2023.03.3


In [2]:
# Functions used in this notebook

def align_structures_to_lowest_energy_and_show(moldict, energy_dict, core_smiles):
    """
    align all structures in "moldict" to the one of the lowest energy
    """
    energy_sorted = sorted(energy_dict.items(), key=lambda x: x[1])
    lowest = energy_sorted[0][0]
    core_lowest = moldict[lowest].GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    
    for key, mol in moldict.items():
        match_mol_to_core = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
        AllChem.AlignMol(mol,moldict[lowest],atomMap=list(zip(match_mol_to_core,core_lowest)))
        
    p = py3Dmol.view(width=400,height=400)
    for key, mol in moldict.items(): 
        mb = Chem.MolToMolBlock(mol)
        p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.15'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p



def from_molfiles_to_moldict(inpfiles, prefix):
    moldict = {}
    for id, inp in enumerate(inpfiles):
        mol = Chem.MolFromMolFile(inp)
        mol = Chem.AddHs(mol)
        name = prefix + str(id)
        moldict[name] = mol
    return moldict
    
def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def make_similarity_matrix(moldict):
    # similarity_matrix only between macrocycles (h2o not taken into account)
    similarity_matrix = {}
    
    for k_1, m_1 in moldict.items():
        for k_2, m_2 in moldict.items():
            if (k_1, k_2) in similarity_matrix.keys() or (k_2, k_1) in similarity_matrix.keys():
                # do not work on the same conformers
                pass
            else:
                if k_1 != k_2:
                    rms = AllChem.GetBestRMS(m_1,m_2)
                    similarity_matrix[(k_1, k_2)] = rms 
                    
    return similarity_matrix


def find_atoms(moldict):
    
    O_h2o     = {}
    H_h2o     = {}        
    O_amide   = {}
    H_amide   = {}
    N_amide   = {}
    N_arom    = {}
        
    for k, m in moldict.items():
        Oa  = []
        Hh  = []
        Ha  = []
        Nar = []
        Na  = []

        for atom in m.GetAtoms():
            
#           remember atom numbering from 0 if you compare with e.g. Avogadro

            # oxygen atoms:
            if atom.GetAtomicNum() == 8:
                if all(x.GetAtomicNum() == 1 for x in atom.GetNeighbors()):
                    O_h2o[k] = atom.GetIdx()
                else:
                    Oa.append(atom.GetIdx())
                    
            # hydrogen atoms:
            if atom.GetAtomicNum() == 1:
                if all(x.GetAtomicNum() == 8 for x in atom.GetNeighbors()):
                    Hh.append(atom.GetIdx())
                elif all(x.GetAtomicNum() == 7 for x in atom.GetNeighbors()):
                    Ha.append(atom.GetIdx())
                    
            # nitrogen atoms:
            if atom.GetAtomicNum() == 7:
                if all(x.GetAtomicNum() == 6 for x in atom.GetNeighbors()):
                    Nar.append(atom.GetIdx())
                else:
                    Na.append(atom.GetIdx())                    
            
            O_amide[k] = Oa
            H_h2o[k]   = Hh
            H_amide[k] = Ha
            N_amide[k] = Na
            N_arom[k]  = Nar
                    
    return O_h2o, O_amide, H_h2o, H_amide, N_amide, N_arom    

def get_distances(moldict, O_h2o, O_amide, H_h2o, H_amide, N_amide, N_arom):
    
    dist_Oh2o_Hamide = {}
    dist_Hh2o_Narom = {}
    dist_Hh2o_Oamide = {}
    dist_Hh2o_Namide = {}
    
    for k, m in moldict.items():
        dm = Chem.Get3DDistanceMatrix(m)
        
        dist1 = []
        for h_idx in H_amide[k]:
            dist1.append(dm[O_h2o[k],h_idx])
        dist_Oh2o_Hamide[k] = sorted(dist1)
        
        dist2 = []
        dist3 = []
        dist4 = []        
        for h_idx in H_h2o[k]:
            for n1_idx in N_arom[k]:
                dist2.append(dm[n1_idx,h_idx])
            for n2_idx in N_amide[k]:
                dist3.append(dm[n2_idx,h_idx])
            for o_idx in O_amide[k]:
                dist4.append(dm[o_idx,h_idx])                

        dist_Hh2o_Narom[k]  = sorted(dist2)
        dist_Hh2o_Namide[k] = sorted(dist3)
        dist_Hh2o_Oamide[k] = sorted(dist4)            
        
    return dist_Oh2o_Hamide, dist_Hh2o_Narom, dist_Hh2o_Namide, dist_Hh2o_Oamide


def find_duplicates_in_sorted_similarity_matrix_noncovalent(similarity_matrix_sorted,
                                                            energy,
                                                            dist_Oh2o_Hamide,
                                                            dist_Hh2o_Narom,
                                                            dist_Hh2o_Namide,
                                                            dist_Hh2o_Oamide):
    
    similarity_thresh = 1.0 # Angstrom
    energy_thresh     = 5   # kcal/mol
    distance_thresh   = 1.0 # Angstrom
    
    to_be_deleted     = []
    
    for pair in similarity_matrix_sorted:
        if pair[1] < similarity_thresh:
            conf1 = pair[0][0]
            conf2 = pair[0][1]
            # now check minimum and maximum distances:
            if (abs(dist_Oh2o_Hamide[conf1][0]  - dist_Oh2o_Hamide[conf2][0]) < distance_thresh and 
                abs(dist_Oh2o_Hamide[conf1][-1] - dist_Oh2o_Hamide[conf2][-1]) < distance_thresh and
                abs(dist_Hh2o_Narom[conf1][0]   - dist_Hh2o_Narom[conf2][0]) < distance_thresh and 
                abs(dist_Hh2o_Narom[conf1][-1]  - dist_Hh2o_Narom[conf2][-1]) < distance_thresh and
                abs(dist_Hh2o_Namide[conf1][0]  - dist_Hh2o_Namide[conf2][0]) < distance_thresh and 
                abs(dist_Hh2o_Namide[conf1][-1] - dist_Hh2o_Namide[conf2][-1]) < distance_thresh and
                abs(dist_Hh2o_Oamide[conf1][0]  - dist_Hh2o_Oamide[conf2][0]) < distance_thresh and 
                abs(dist_Hh2o_Oamide[conf1][-1] - dist_Hh2o_Oamide[conf2][-1]) < distance_thresh):
                
                
                # finally check energies and remove the one with the higher:
                if abs(energy[conf1] - energy[conf2]) < energy_thresh:
                    if energy[conf1] < energy[conf2]:
                        to_be_deleted.append(conf2)
                    else:
                        to_be_deleted.append(conf1)
            else:
                # similarity_matrix_b_sorted is sorted, so here we would already start looping over 
                # pairs for which rmsd is > threshold 
                # and we do not need to do this, so break
                break

    return to_be_deleted


## Example 1

In [3]:
# first, we select input files with molecular geometries (note that this example uses sdf format)

inps_m1_h2o_in_rdkit        = glob.glob('results_starting_from_m1_h2o_in/*.sdf')
inps_m1_h2o_out_rdkit       = glob.glob('results_starting_from_m1_h2o_out/*.sdf')
inps = inps_m1_h2o_in_rdkit + inps_m1_h2o_out_rdkit

In [4]:
# then, we write molecules to a dictionary;
# this is handy, as we can use the key to keep track of names (here: "name" variable)
moldict_m1_h2o_in_rdkit  = from_molfiles_to_moldict(inps_m1_h2o_in_rdkit, "m1_h2o_in_conformers_from_rdkit_")
moldict_m1_h2o_out_rdkit = from_molfiles_to_moldict(inps_m1_h2o_out_rdkit, "m1_h2o_out_conformers_from_rdkit_")
moldict = moldict_m1_h2o_in_rdkit | moldict_m1_h2o_out_rdkit
print("We have {} conformers to consider".format(len(moldict)))

We have 5 conformers to consider


In [5]:
# grep conformer energies from sdf files
e_m1_h2o_in_rdkit  = grep_energies_from_sdf_outputs(inps_m1_h2o_in_rdkit)
e_m1_h2o_out_rdkit = grep_energies_from_sdf_outputs(inps_m1_h2o_out_rdkit)
energies=e_m1_h2o_in_rdkit | e_m1_h2o_out_rdkit

In [6]:
# We can have a look at all generated conformers.
# Here, we align all structures to the one of the lowest energy.
# We first need to set the "core" - a part of molecule, which we wish to be most aligned (rmsd-wise) among all the structures;
# this, we give in smiles format
mol_smiles = 'O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'
core_smiles = 'n1ccccc1'

In [7]:
p = align_structures_to_lowest_energy_and_show(moldict, energies, core_smiles)
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
# Let's now do some post-analysis. Our goal is to (1) prune the set of conformers and (2) add some similarity measures to our analysis.

# For that, we first calculate the similarity matrix between all pairs of conformers and sort its elements
# from the lowest values (the most similar structures) to the largest values (the most different structures)
similarity_matrix = make_similarity_matrix(moldict)
similarity_matrix_sorted = sorted(similarity_matrix.items(), key=lambda x: x[1])
similarity_matrix_sorted

[(('m1_h2o_in_conformers_from_rdkit_0', 'm1_h2o_out_conformers_from_rdkit_0'),
  0.733951463091113),
 (('m1_h2o_in_conformers_from_rdkit_0', 'm1_h2o_out_conformers_from_rdkit_1'),
  0.7407696948403357),
 (('m1_h2o_in_conformers_from_rdkit_0', 'm1_h2o_out_conformers_from_rdkit_2'),
  0.812910819741077),
 (('m1_h2o_in_conformers_from_rdkit_1', 'm1_h2o_out_conformers_from_rdkit_1'),
  0.8529479915629845),
 (('m1_h2o_out_conformers_from_rdkit_0', 'm1_h2o_out_conformers_from_rdkit_1'),
  0.8696061915931911),
 (('m1_h2o_in_conformers_from_rdkit_1', 'm1_h2o_out_conformers_from_rdkit_0'),
  0.8746202345780906),
 (('m1_h2o_in_conformers_from_rdkit_0', 'm1_h2o_in_conformers_from_rdkit_1'),
  0.9645476311420325),
 (('m1_h2o_out_conformers_from_rdkit_1', 'm1_h2o_out_conformers_from_rdkit_2'),
  1.0416603999400982),
 (('m1_h2o_out_conformers_from_rdkit_0', 'm1_h2o_out_conformers_from_rdkit_2'),
  1.136199837621698),
 (('m1_h2o_in_conformers_from_rdkit_1', 'm1_h2o_out_conformers_from_rdkit_2'),
  1.

In [9]:
# Then, we select "key atoms" in the systems we study, such as atoms that are/can be involved in non-covalent interactions.
# In this case, we look for
O_h2o, O_amide, H_h2o, H_amide, N_amide, N_arom = find_atoms(moldict)

In [10]:
dist_Oh2o_Hamide, dist_Hh2o_Narom, dist_Hh2o_Namide, dist_Hh2o_Oamide = get_distances(moldict,
                                                                                      O_h2o=O_h2o,
                                                                                      O_amide=O_amide,
                                                                                      H_h2o=H_h2o,
                                                                                      H_amide=H_amide,
                                                                                      N_amide=N_amide,
                                                                                      N_arom=N_arom)

#print(dist_Oh2o_Hamide, dist_Hh2o_Narom, dist_Hh2o_Namide, dist_Hh2o_Oamide)

In [11]:
# 2. remove duplicates:
# for all pairs of structures, for which the similarity value is lower than the threshold ("similarity_thresh"), 
# compare distances of H2O - macrocycle; then finally compare energies
to_be_deleted = find_duplicates_in_sorted_similarity_matrix_noncovalent(
    similarity_matrix_sorted,
    energies,
    dist_Oh2o_Hamide,
    dist_Hh2o_Narom,
    dist_Hh2o_Namide,
    dist_Hh2o_Oamide)

for mol in to_be_deleted:
    print("to_be_deleted: ", mol)
    to_be_deleted_keys = list(k for k in similarity_matrix.keys() if mol in k)
    for k in to_be_deleted_keys:
        del similarity_matrix[k]
    moldict.pop(mol, None)
    energies.pop(mol, None)
    
print("Number of conformers after removal of duplicates: ", len(moldict))    

to_be_deleted:  m1_h2o_out_conformers_from_rdkit_0
to_be_deleted:  m1_h2o_out_conformers_from_rdkit_1
to_be_deleted:  m1_h2o_in_conformers_from_rdkit_0
to_be_deleted:  m1_h2o_out_conformers_from_rdkit_1
to_be_deleted:  m1_h2o_out_conformers_from_rdkit_1
to_be_deleted:  m1_h2o_out_conformers_from_rdkit_0
to_be_deleted:  m1_h2o_in_conformers_from_rdkit_0
Number of conformers after removal of duplicates:  2


In [12]:
print(len(moldict))

2


In [13]:
print("All the selected conformers (aligned), pre-optimized with MM methods:")
p = align_structures_to_lowest_energy_and_show(moldict, energies, core_smiles)
p.show()

All the selected conformers (aligned), pre-optimized with MM methods:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
#Write the selected conformers' names to the list "list_selected_conformers_from_rdkit". 
# It will be used to generate inputs for quantum chemistry calculations:

with open("/home/gosia/projects/websites/qchemlab-embedding/scripts/analysis/conformer_generation/list_selected_conformers_from_rdkit", "w") as f:
    for key, mol in moldict.items():
        f.write(key+"\n")

energy_sorted = sorted(energies.items(), key=lambda x: x[1])
with open("/home/gosia/projects/websites/qchemlab-embedding/scripts/analysis/conformer_generation/detailed_list_selected_conformers_from_rdkit", "w") as f:
    for pair in energy_sorted:
        f.write("{0:30}   {1}\n".format(pair[0], pair[1])) 